In [1]:
# 单个cell的前向传播过程
# 两个输入： x_t, s_prev, 以及 parameters

In [4]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def rnn_cell_forward(x_t, s_prev, parameters):
    """
    x_t: 当前t时刻的序列输入
    s_prev: 上一个时刻cell的隐层状态输入
    parameters: cell中的参数
    return: 隐层输出 s_next, 当前这层输出 out_pred
    """
    
    # 参数parameters
    U = parameters["U"]
    W = parameters["W"]
    V = parameters["V"]
    b_a = parameters["b_a"]
    b_y = parameters["b_y"]
    
    # 隐层输出计算
    s_next = np.tanh(np.dot(U, x_t) + np.dot(W, s_prev) + b_a)
    
    # 计算cell的输出
    out_pred = softmax(np.dot(V, s_next) + b_y)
    
    # 记录每一层的值，用于反向传播计算使用 
    cache = (s_next, s_prev, x_t, U, V, W, out_pred)
    
    return s_next, out_pred, cache

In [15]:
# 初始化所有cell的S，用于保存所有cell的隐层结果
# 初始化所有cell的输出y，保存所有输出结果

def rnn_forward(x, s0, parameters):
    
    """
    x: 输入序列，形状(m, 1,T), T为序列长度
    s0: 初始状态输入
    parameters: 所有cell的共享参数，U,W,V,ba, by
    return: s, y, caches

    """
    caches = []
    
    # 获取序列的长度，时刻数
    m, _, T = x.shape # [3,1,4] 的shape means 3rows, 1 line each row, 4 elements each line 
    [1,10000,3]
    
    [[1,10000],
     [1,10000],
     [1,10000]]
    
    # 获取输入的N，定义隐层输出大小状态
    m,n = parameters["V"].shape # m?
    
    # 获取s0的值，保存到s_next里面，以便于前向传播传入到cell.
    s_next = s0
    
    # 定义s,y并保留所有cell的隐层状态以及输出
    s = np.zeros((n, 1, T)) 
    
    y = np.zeros((m,1,T))
    
    # 循环对每一个cell进行前向传播计算
    for t in range(T):
        #对于t时刻cell进行输出
        s_next, out_pred, cache = rnn_cell_forward(x[:,:,T], s_next, parameters)
        
        #放入数组当中
        s[:,:,t] = s_next
        
        y[:,:,t] = out_pred

        # 放入所有的缓存到列表当中
        caches.append(cache)
        
    return s, y, caches

In [2]:
# 构建反向传播过程

def rnn_cell_backward(ds_next, cache):
    
    """
    每个cell的右边输入梯度
    ds_next：s_next的梯度值
    cache: 当前cell的缓存
    return: 该cell的六个gradients
    """
    
    #获取cache中的缓存值以及参数
    (s_next, s_prev, x_t, parameters) = cache
    
    #根据公式进行反向传播计算
    #1. 计算tanh的导数
    dtanh = (1 - s_next ** 2) * ds_next
    
    #2. 计算U的导数（梯度值）
    dU = np.dot(dtanh, x_t.T)
    
    #3. 计算W的梯度值
    dW = np.dot(dtanh, s_prev.T)
    
    #4. 计算ba的梯度值
    #保持计算之后U的维度不变
    dba = np.sum(dtanh, axis = 1, keepdims = 1)
    
    #5. 计算x_t的导数
    dx_t = np.dot(U.T, dtanh)
    
    #6. 计算s_prev的导数
    ds_prev = np.dot(W.T, dtanh)
    
    #把所有的导数保存到字典当中返回
    gradients = {"dtanh": dtanh, "dU":dU, "dW":dW, "dba": dba, "dx_t":dx_t, "ds_prev":ds_prev}
    
    return gradients

In [3]:
# 所有cell的反向传播过程

def rnn_backward(ds,caches):
    
    """
    ds: 每个时刻的损失对于s的梯度值，假设是已知的(5,1,4)
    caches: 每个cell的输出值
    """
    
    #取出caches当中的值
    (s1, s0, x_1, parameters) = cache[0]
    
    #获取输入数据的总共序列长度
    n, _, T = ds.shape
    m, _ = x_1.shape
    
    # 存储所有一次更新后的参数的梯度
    dU = np.zeros((n,m))
    dW = np.zeros((n,n))
    dba = np.zeros((n,1))
    
    # 初始化一个为0的s第二部分梯度值
    ds_prevt = np.zeros((n,1))
    
    # 保存其他不需要更新的梯度
    dx = np.zeros((m, 1, T))
    
    # 循环从后往前进行梯度计算
    for t in reversed(range(T)):
        
        # 从3时刻开始, 2,1,0时刻的梯度由两个部分组成
        gradients = rnn_cell_backward(ds[:, :, t] + ds_prevt, caches[t])
        
        ds_prevt = gradients["ds_prev"]
        
        # U, W, ba, x_t, s_prev梯度， 共享参数需要相加
        dU += gradients["dU"]
        dW += gradients["dW"]
        dba += gradients["dba"]
        
        # 保存每一层的x_t, s_prev的梯度值
        dx[:, :, t] = gradients["dx_t"]
    
        # 返回所有更新参数的梯度以及其他变量的梯度值
        gradients = {"dU": dU, "dW": dW, "dba": dba, "dx": dx }
        
        return gradients
    
    

In [1]:
import numpy as np

arr = np.array([1, 3, 7, 2, 6, 4])
argmax_value = np.argmax(arr)

print("Array:", arr)
print("Index of the maximum value:", argmax_value)
print("Maximum value:", arr[argmax_value])

Array: [1 3 7 2 6 4]
Index of the maximum value: 2
Maximum value: 7
